# First part Project Week3: Website Scraping
#### Several assumptions
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

Using the package BeautifulSoup, we take the first table found on the webpage and put it in a list

In [1]:
from bs4 import BeautifulSoup
import requests

# get html source
html_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_source, 'lxml')

# get the table from the page
table = soup.find('table')

tablelist = []
for element in table.find_all('tr'):
    tablelist.append(element.text.split('\n')[1:4])

Create dataframe and clean the data

In [22]:
# create dataframe
import pandas as pd

df = pd.DataFrame(tablelist)
df.columns = df.iloc[0]
df = df.loc[1:]

# Dropping cells with borough not assigned
na_borough = df[df['Borough'] == 'Not assigned'].index
df.drop(na_borough, axis=0, inplace=True)

# Assign neighbourhood
na_neighbourhood = df[df['Neighbourhood'] == 'Not assigned'].index
for idx in na_neighbourhood:
    df.loc[idx,'Neighbourhood'] = df.loc[idx]['Borough']

df = df.reset_index(drop=True)

Combine neighbourhood with the same postcode

In [23]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [24]:
df.shape

(103, 3)